In [1]:
import os
import pandas as pd

os. chdir("c:\\Users\\simon\\OneDrive\\Dokumente\\UNILU\\3 - HS24\\1 MA\\business-reports-nlp")

## Create the first sample (n = 250)

In [ ]:
texts = pd.read_csv('data/paragraphs.csv', encoding = 'utf-8')

sample = texts.sample(n=250)


def remove_illegal_characters(s):
    return ''.join([char for char in s if char.isprintable()])

# Apply the function to each string column
for col in sample.select_dtypes(include=['object']):
    sample[col] = sample[col].apply(remove_illegal_characters)

sample.to_excel('data/sample.xlsx', index=False, engine='openpyxl')

## Create the second sample (n = 1250)

In [ ]:
sample = pd.read_excel('data/sample.xlsx')
sample = sample.drop(['index', 'category', 'key', 'note'], axis=1)

# Define column which contains the matching entries
common_column = 'content'

# Merge the two DataFrames based on the common column
merged_df = texts.merge(sample, on=common_column, how='left', indicator=True)

# Keep only the rows that are in df1 but not in df2 based on the indicator column
filtered_df1 = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column
filtered_df1 = filtered_df1.drop(columns=['_merge'])

# Now, filtered_df1 contains the rows from df1 where the 'common_column' values
# are not identical to the values in df2's 'common_column'.

new_sample = filtered_df1.sample(n=1250, random_state=42)

Double-check whether second sampling worked.

In [ ]:
if sample['content'].isin(new_sample['content']).any():
    print("There are identical entries between df1 and df2.")
else:
    print("There are no identical entries between df1 and df2.")

Save as xlsx.

In [ ]:
def remove_illegal_characters(s):
    if isinstance(s, str):
        return ''.join([char for char in s if char.isprintable()])
    else:
        return s

# Apply the function to each string column
for col in new_sample.select_dtypes(include=['object']):
    new_sample[col] = new_sample[col].apply(remove_illegal_characters)

new_sample.to_excel('data/sample2.xlsx', index=False, engine='openpyxl')

## Further split sample2

find empty rows from sample2

In [3]:
sample2 = pd.read_excel('data/sample2.xlsx')

# Filtering rows where 'classification' is NaN
empty_classification = sample2[pd.isna(sample2['classification'])]

split empty rows into df with n = 250 and another df with n = 500.

In [8]:
# Randomly sample 250 cases
sample3 = empty_classification.sample(n=250, random_state=42)

# Create a DataFrame with the remaining cases
sample4 = empty_classification.drop(sample3.index)

save to xlsx.

In [9]:
sample3.to_excel('data/sample3.xlsx', index=False, engine='openpyxl')

sample4.to_excel('data/sample4.xlsx', index=False, engine='openpyxl')

## Final sampling
into:
- training
- validation
- classifiction

In [2]:
# complete
complete = pd.read_csv("data/paragraphs.csv")
complete = complete.drop_duplicates(subset='content', keep='first')

In [3]:
# training
training = pd.read_excel("data/fine-tuning v1/_old/20240101_training.xlsx")
columns_to_merge = ['year', 'file_name', 'source', 'file_path']
training = pd.merge(training, complete[columns_to_merge + ['content']], on='content', how='left')
training = training.sample(frac=1).reset_index(drop=True)

In [45]:
# validation
validation = pd.read_excel("data/fine-tuning v1/validation.xlsx")
validation = validation.rename({'year_x': 'year', 'file_name_x': 'file_name', 'file_path_x': 'file_path', 'source_x': 'source', 'corrected': 'classification'}, axis=1)
validation = validation.drop(['index', 'Spalte1'], axis=1)

In [46]:
# classification
content_list = pd.concat([training['content'], validation['content']]).tolist()
classification = complete[~complete['content'].isin(content_list)]

# manually remove rows which contain the following entries within the "content" column
to_remove = validation[~validation['content'].isin(complete['content'])]['content'].to_list()
to_remove.extend(training[~training['content'].isin(complete['content'])]['content'].to_list())
to_remove = pd.DataFrame(to_remove, columns=['content'])

Now for some final text cleaning and finally exporting the data

In [5]:
import re

def clean_text_column(df, column_name):
    """
    Cleans the text entries in the specified column of a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the text column.
    column_name (str): The name of the column to be cleaned.
    
    Returns:
    pd.DataFrame: The DataFrame with cleaned text in the specified column.
    """
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df_cleaned = df.copy()

    # Replace "- " with nothing
    df_cleaned.loc[:, column_name] = df_cleaned[column_name].str.replace("- ", "", regex=False)
    
    # Replace multiple spaces with a single space
    df_cleaned.loc[:, column_name] = df_cleaned[column_name].apply(lambda x: re.sub(' +', ' ', x))
    
    return df_cleaned

In [48]:
training = clean_text_column(training, 'content')
validation = clean_text_column(validation, 'content')
classification = clean_text_column(classification, 'content')
to_remove = clean_text_column(to_remove, 'content')

In [49]:
training.to_csv('data/fine-tuning v1/training.csv', index=False)
validation.to_csv('data/fine-tuning v1/validation.csv', index=False)
classification.to_csv('data/fine-tuning v1/classification.csv', index=False)
to_remove.to_csv('data/fine-tuning v1/to_remove.csv', index=False)

C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\internals\blocks.py:2540: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


# Sampling for qualitative analysis

In [2]:
df = pd.read_csv('data/final.csv')

In [11]:
# Define the range of years
years = range(2007, 2024)

# Define your classifications
classifications = ["Interactions between Payments Players", "Data Monetization", 
                   "Growth of Digital Payments", "Delivering the Payments Experience", 
                   "Sustainability", "Other"]

# Initialize quotas for each combination of classification and year
quotas = {(classification, year): 10 for classification in classifications for year in years}

# Function to apply the quota sampling
def apply_quota_sampling(group, quotas):
    classification, year = group.name
    quota = quotas.get((classification, year), 0)
    return group.sample(n=min(quota, len(group)))

# Sample based on quotas
sampled_dfs = df.groupby(['classification', 'year']).apply(apply_quota_sampling, quotas=quotas)

# Reset index if needed
sampled_dfs = sampled_dfs.reset_index(drop=True)

In [12]:
sampled_dfs['classification'].value_counts()

classification
Growth of Digital Payments               170
Interactions between Payments Players    170
Other                                    167
Data Monetization                        161
Sustainability                           139
Name: count, dtype: int64

In [14]:
sampled_dfs['source'].value_counts()

source
capgemini    392
mckinsey     242
bcg           91
bain          43
accenture     39
Name: count, dtype: int64

In [17]:
def remove_illegal_characters(s):
    if isinstance(s, str):
        return ''.join([char for char in s if char.isprintable()])
    else:
        return s

# Apply the function to each string column
for col in df.select_dtypes(include=['object']):
    sampled_dfs[col] = sampled_dfs[col].apply(remove_illegal_characters)

sampled_dfs.to_excel('data/qualitative/quota-sampling.xlsx')